In [7]:
import torch
import torch.nn as nn


class ScratchLSTM(nn.Module):

    def __init__(self, input_size, hidden_size):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size

        # 入力 x_t に掛ける重み
        self.W_x = nn.Parameter(torch.randn(4 * hidden_size, input_size)) # 4つのゲート（i,f,g,o）を一括計算するため4*H
        # 直前の隠れ状態 h_{t-1} に掛ける重み
        self.W_h = nn.Parameter(torch.randn(4 * hidden_size, hidden_size))
        # バイアス（4ゲート分）
        self.b = nn.Parameter(torch.zeros(4 * hidden_size))

    def forward(self, x):
        B, T, D = x.shape # B:バッチサイズ、 T:時系列長、 D:入力次元
        H = self.hidden_size

        # 初期の隠れ状態 h_0 とセル状態 c_0, shape: (B, H)
        h_prev = torch.zeros(B, H, device=x.device)
        c_prev = torch.zeros(B, H, device=x.device)

        outputs = []

        # 時系列方向にループ（RNNの本質）
        for t in range(T):

            # 時刻 t の入力, shape: (B, D)
            x_t = x[:, t, :]

            # ゲートを一括で計算
            # shape: (B, 4H)
            gates = (
                x_t    @ self.W_x.T +   # 入力からの影響
                h_prev @ self.W_h.T +   # 直前の隠れ状態からの影響
                self.b               # バイアス
            )

            # ゲートを4つに分割
            forget_gate = torch.sigmoid(gates[:, 0:H])

            input_gate = torch.sigmoid(gates[:, H:2*H])
            cell_candidate = torch.tanh(gates[:, 2*H:3*H])

            output_gate = torch.sigmoid(gates[:, 3*H:])
            
            
            c_next = (forget_gate * c_prev) + (input_gate * cell_candidate)
            h_next = output_gate * torch.tanh(c_next)

            outputs.append(h_next)

        # outputs は list[(B,H)] × T
        # → (B, T, H) に変換
        y = torch.stack(outputs, dim=1)

        return y
            

In [8]:
import torch

# 再現性のため乱数固定
torch.manual_seed(0)

# -----------------------
# ダミー入力の作成
# -----------------------
B = 2   # バッチサイズ
T = 4   # 時系列長
D = 3   # 入力次元
H = 5   # 隠れ状態次元

# (B, T, D)
x = torch.randn(B, T, D)
print(x.shape)

# -----------------------
# LSTM インスタンス作成
# -----------------------
lstm = ScratchLSTM(input_size=D, hidden_size=H)

# -----------------------
# 順伝播
# -----------------------
y = lstm(x)

# -----------------------
# 出力確認
# -----------------------
print("input shape :", x.shape)   # (B, T, D)
print("output shape:", y.shape)   # (B, T, H)
print("output:", y)

torch.Size([2, 4, 3])
input shape : torch.Size([2, 4, 3])
output shape: torch.Size([2, 4, 5])
output: tensor([[[-0.1000, -0.1228,  0.0303, -0.0380,  0.4951],
         [ 0.1861,  0.3670, -0.0670, -0.0375, -0.0896],
         [-0.1709, -0.0653,  0.0118, -0.3139, -0.4131],
         [ 0.2190,  0.2922, -0.1719,  0.0331, -0.0157]],

        [[ 0.1672,  0.2144, -0.1303,  0.0580,  0.0098],
         [ 0.2011,  0.4139, -0.1127, -0.1024, -0.0811],
         [ 0.0273,  0.1772, -0.4318, -0.0295, -0.0838],
         [-0.2769, -0.0538,  0.4191,  0.2572,  0.3438]]],
       grad_fn=<StackBackward0>)
